In [ ]:
#import dependencies
import pymongo
from bs4 import BeautifulSoup as bs
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from sqlalchemy import create_engine
from config import password, host
import requests
import pandas as pd

# Scraping NFL Salary Data using Pandas

In [ ]:
#enable chrome browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [ ]:
salary_url = 'https://www.pro-football-reference.com/players/salary.htm'
nfl_wiki_url = 'https://en.wikipedia.org/wiki/Wikipedia:WikiProject_National_Football_League/National_Football_League_team_abbreviations'

In [ ]:
#read table from url
tables = pd.read_html(salary_url)
tables

In [ ]:
#create dataframe based on 1st table
salary_df = tables[0]
salary_df.head()

## Cleaning the Salary DataFrame

In [ ]:
#rename columns
salary_df = salary_df.rename(columns={'Rk': 'Salary_Rank', 'Pos':'Player_Position', 'Tm': 'Abrv' })

#'Rank' column currently contains blank values when salaries are tied
#this line of code will fill the blank rank value with the value directly above
salary_df['Salary_Rank'] = salary_df['Salary_Rank'].fillna(method='ffill')

#fill blank positions with 'NULL' text string
salary_df['Player_Position'] = salary_df['Player_Position'].fillna('NULL')

#Split out Player Name and store in a temporary dataframe
temp_df = salary_df['Player'].str.split(' ', n=1, expand=True)

#add first name and last name to original salary dataframe
salary_df['First_Name'] = temp_df[0]
salary_df['Last_Name'] = temp_df[1]

#remove '$' and ',' characters from Salary 
salary_df['Salary'] = salary_df['Salary'].str.replace('$','')
salary_df['Salary'] = salary_df['Salary'].str.replace(',','')

#convert salary datatype to int
salary_df['Salary'] = salary_df['Salary'].astype(int)

#change abbreviations to official abbreviations
salary_df = salary_df.replace({'TAM':'TB', 'OAK': 'LV', 'NOR': 'NO', 'SFO': 'SF', 
                                'GNB': 'GB', 'KAN': 'KC', 'NWE': 'NE'})

#create unique ID using player name and position
salary_df['uid'] = salary_df['Player'] + salary_df['Player_Position']

#rearrange columns to finish cleaning the salary dataframe
columns = ['Salary_Rank', 'uid','Player', 'First_Name', 'Last_Name', 'Player_Position', 'Abrv', 'Salary']
salary_df = salary_df[columns]

salary_df.head()

# Pull NFL Abbreviation Mapping from Wikipedia

In [ ]:
tables = pd.read_html(nfl_wiki_url)

#create dataframe based on 1st table
nfl_mapping = tables[0]

#assign 1st row as headers
nfl_mapping.columns = nfl_mapping.iloc[0]
nfl_mapping = nfl_mapping[1:]

#rename abbreviation columns
nfl_mapping = nfl_mapping.rename(columns={'Abbreviation/Acronym':'Abrv', 'Franchise':'Team'})

nfl_mapping.head()

## Merge Full NFL Name to Salary Table

In [ ]:
#merge salary df with nfl mapping
salary_df = pd.merge(salary_df, nfl_mapping, on = 'Abrv', how='left')

In [ ]:
#rearrange columns
columns = ['Salary_Rank', 'uid', 'Player', 'First_Name', 'Last_Name', 'Player_Position', 'Abrv', 'Team', 'Salary']
salary_df = salary_df[columns]

#sort df by rank
salary_df = salary_df.sort_values(by=['Salary_Rank'])

#make columns lowercase
salary_df.columns = salary_df.columns.str.lower()

salary_df.head()

In [ ]:
#save csv just for reference
salary_df.to_csv('salaries.csv', index=False)
salary_df

# NFL Players by Draft Position

In [ ]:
url = 'http://www.drafthistory.com/index.php/years/'

In [ ]:
rounds = []
years = list(range(1998,2020))

for year in years:
    request = requests.get(f'{url}{year}')
    draft = pd.read_html(request.text, header=1)
    draft = draft[0]
    rounds.append(draft)

In [ ]:
draft1998 = rounds[0]
draft1999 = rounds[1]
draft2000 = rounds[2] 
draft2001 = rounds[3] 
draft2002 = rounds[4]
draft2003 = rounds[5]
draft2004 = rounds[6]
draft2005 = rounds[7]
draft2006 = rounds[8]
draft2007 = rounds[9]
draft2008 = rounds[10] 
draft2009 = rounds[11] 
draft2010 = rounds[12]
draft2011 = rounds[13]
draft2012 = rounds[14]
draft2013 = rounds[15]
draft2014 = rounds[16]
draft2015 = rounds[17]
draft2016 = rounds[18]
draft2017 = rounds[19]
draft2018 = rounds[20]
draft2019 = rounds[21]

In [ ]:
draft1998['Year'] = '1998'
draft1999['Year'] = '1999'
draft2000['Year'] = '2000' 
draft2001['Year'] = '2001' 
draft2002['Year'] = '2002'
draft2003['Year'] = '2003'
draft2004['Year'] = '2004'
draft2005['Year'] = '2005'
draft2006['Year'] = '2006'
draft2007['Year'] = '2007'
draft2008['Year'] = '2008'
draft2009['Year'] = '2009'
draft2010['Year'] = '2010'
draft2011['Year'] = '2011'
draft2012['Year'] = '2012'
draft2013['Year'] = '2013'
draft2014['Year'] = '2014'
draft2015['Year'] = '2015'
draft2016['Year'] = '2016'
draft2017['Year'] = '2017'
draft2018['Year'] = '2018'
draft2019['Year'] = '2019'

In [ ]:
all_drafts = [draft1998, draft1999, draft2000, draft2001, draft2002, draft2003, draft2004, draft2005, draft2006, 
              draft2007, draft2008, draft2009, draft2010, draft2011, draft2012, draft2013, draft2014, draft2015, 
              draft2016, draft2017, draft2018, draft2019]

all_drafts = pd.concat(all_drafts).reset_index(drop=True)

all_drafts.head(50)

In [ ]:
all_drafts['Round'] = all_drafts['Round'].fillna(method='ffill')

all_drafts = all_drafts.rename(columns={"Pick": "Round_Pick", "Player": "Overall_Pick"})

# all_drafts.loc[all_drafts['Name'] == 'Adrian Peterson']
small_info = all_drafts[['Name', 'Position']]
small_info = small_info[small_info.duplicated()]
small_info

In [ ]:
url = 'https://www.pro-football-reference.com/years/2019/scrimmage.htm'

In [ ]:
tables = pd.read_html(url, header=[1])
stats2019 = tables[0]
stats2019['Player'] = stats2019['Player'].map(lambda x: x.rstrip('+*'))
stats2019

In [ ]:
stats2019 = stats2019.replace({'TAM':'TB', 'OAK': 'LV', 'NOR': 'NO', 'SFO': 'SF', 
                                'GNB': 'GB', 'KAN': 'KC', 'NWE': 'NE'})
stats2019

In [ ]:
stats2019 = stats2019[['Player', 'Tm', 'Age', 'Pos', 'G', 'Touch', 'Y/Tch', 'YScm', 'RRTD', 'Fmb', 'Rec']]

stats2019

In [ ]:
stats2019 = stats2019.rename(columns={'Tm':'Team', 'Pos': 'Position', 'G': 'Games', 'Y/Tch': 'Yards/Touch', 
                                'YScm': 'Total_Yards', 'RRTD': 'Touchdowns', 'Fmb': 'Fumbles', 'Rec':'Receptions', 'Touch': 'Touches'})

stats2019

In [ ]:
stats2019 = stats2019.loc[stats2019['Total_Yards'] != 'YScm']

stats2019

# Extract 2019 NFL season standings using Pandas

In [ ]:
# Pull seaonson standings table data using Pandas
url = 'http://www.playoffstatus.com/nfl/nflpostseasonprob.html'
tables = pd.read_html(url, header=[1])

In [ ]:
# Store correct table in variable
nfl_2019_standings_df = tables[0]
nfl_2019_standings_df

In [ ]:
# Rename columns correct 49ers name
nfl_2019_standings_df = nfl_2019_standings_df[['Team','Conference Conf','W','L','T']]
nfl_2019_standings_df = nfl_2019_standings_df.rename(columns={"Conference Conf": "Conference", "W": "Wins", "L":"Losses","T":"Ties"})
nfl_2019_standings_df.at[1,'Team'] = '49ers'
nfl_2019_standings_df

In [ ]:
# Pull NFL team abbreviation data using pandas
url = 'https://en.wikipedia.org/wiki/Wikipedia:WikiProject_National_Football_League/National_Football_League_team_abbreviations'
tables = pd.read_html(url)

In [ ]:
# Clean up abbreviation table 
nfl_abbrev_df = tables[0]
nfl_abbrev_df = nfl_abbrev_df.drop(0)
nfl_abbrev_df = nfl_abbrev_df.rename(columns={0: "Abbreviation", 1: "Team"})
nfl_abbrev_df

In [ ]:
# Split and get last part of team name
nfl_abbrev_df["Team"] = nfl_abbrev_df["Team"].str.split().str[-1]
nfl_abbrev_df.head()

In [ ]:
# Merge season standings table and abbreviation table on team name
nfl_2019_standings_df = pd.merge(nfl_2019_standings_df, nfl_abbrev_df, how='inner', on='Team')
nfl_2019_standings_df.head()

# Connect and Load Data to Postgres Database

In [ ]:
#create postgres connection string
rds_connection_string = f"postgres:{password}@localhost:{host}/NFL"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
#view engine table names
engine.table_names()

In [ ]:
#load salary df to salaries table
salary_df.to_sql(name='salaries', con=engine, if_exists='append', index=False)

In [ ]:
#preview data loaded from table
pd.read_sql_query('select * from salaries limit 10', con=engine).head()

In [ ]:
connection_string = "postgres:R4$hcr4ft13@localhost:5432/NFL_Draft"
engine = create_engine(f'postgresql://{connection_string}')

engine.table_names()

In [ ]:
all_drafts.to_sql(name='drafts', con=engine, if_exists='append', index=False)

In [ ]:
connection_string = f"postgres:{pswd}@localhost:5432/NFL_Draft"
engine = create_engine(f'postgresql://{connection_string}')

engine.table_names()

In [ ]:
stats2019.to_sql(name='player_stats', con=engine, if_exists='append', index=False)

In [ ]:
new_customer_data_df.to_sql(name='nfl_2019_standings', con=engine, if_exists='append', index=False)